# **Instalción de librerias**

In [1]:
from predict_noise import process_media
import os
folder_test = 'test'
files_to_test = os.listdir(folder_test)
case1 = False
case2 = False

percentage_layers_case_1  = 0.5 if case1 else 0
percentage_layers_case_2 = 0.5 if case2 else 0
# Con este parámetro seleccionamos la zona en la que queremos afectar a las activacionies o pesos
# es decir, slices = [slice_i, n_slices], con esto dividimos todas las capas en n_slices y afectamos a las capas que esten en slice_i
'''
|slice 0 | slice 1 | slice 2 | slice 3 | slice 4 | 
'''
interval_1 = (-8, 2)
interval_2 = (-8, 2)
slices_case1 = [0,1] 
slices_case2 = [0,1]
percentage_tensors_1 = 0.05
percentage_tensors_2 = 0.002


folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
process_media(folder_test, files_to_test, case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1= percentage_tensors_1,
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2,
                folder_model=folder_model, model_name=model_name)


=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar
[1/22] - Processing: test_video2_large_images.mp4
Output video path: /home/jovyan/Carlos_Gonzalez/YOLO/tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/test_noise/test_video2_large_images_predicted.mp4
Average inference time per frame: 0.625237 seconds
[2/22] - Processing: babb0ef2-ef2d-4cab-b3e2-230ae2418cdc_1024.jpg
Inference large image, Time: 0.11321258544921875 seconds, FPS: 8.832940225082552
Average inference time per tile: 0.001902 seconds
Image saved in: /home/jovyan/Carlos_Gonzalez/YOLO/tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/test_noise/babb0ef2-ef2d-4cab-b3e2-230ae2418cdc_1024_predicted.jpg
[3/22] - Processing:

In [1]:
!pip install torchsummary
!pip install torchmetrics
!pip install torchvision
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 13.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 25.7 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 13.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 13.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.18.1
    Uninstalling nvidia-nccl-cu12-2.18.1:
      Successfully uninstalled nvidia-nccl-cu12-2.18.1
  Attempting un

# **Testing Noise**

Uno de los objetivos de este proyecto es la posterior implementación de estos modelos en una FPGA,  con el fin de que sea capaz de resolver problemas de detección desde un satélite.

Se sabe que los modelos en una FPGA están expuestos a ruido, este ruido puede afectar en cualquier parte del modelo, se han considerado los siguientes casos:
* **Caso 1**: se pretende simular el ruido que puede ocurrir dentro del modelo, es decir, entre las conexiones de capas. Fenómeno conocido como SEU.
* **Caso 2**: se simula el ruido que puede afectar a los pesos del modelo debido a las fluctuaciones que pueden surgir en el almacenamiento de estos. Fenómeno conocido como SEL.

Para el caso 1 y 2 de análisis de ruido se ha considerado el caso, de que en caso de que exista ruido este se produce un  bit-flip, en algún punto aleatorio de los pesos o activaciones.

En este ***Notebook*** se pretende analizar diferentes casos, de como afecta los diferentes ruidos de manera aislada. Los resutlados obtenidos son guardados en la carpeta correspondiente del modelo que está siendo ejecutado, en la subcarpeta de ***results_noise***, en el que son analizadas las métricas de loss y mAP50.

Como en los ***Notebooks*** anteriores, la ejecución de estos códigos hay que tener en cuenta que dependen directamente del fichero de configuración ***config.py***, para la selección del modelo. En este cuaderno a diferencia de los otros no ejecutamos el código con un !python eval_noise.py, si no que hacemos un import, por comodidad de este código y poder evaluar los diferentes casos. Siendo así el código, si quieres probar diferentes modelos con el fichero ***config.py*** hay que reiniciar el kernel despues de modificar el fichero.

Si se quiere realizar las pruebas de **noise.py** sobre los modelos optimizados se tiene que especificar la ruta del modelo optimizado aunque tiene que tener coherencia con **config.py**

In [ ]:
!python eval_noise.py

# Análisis para el modelo con precisión máxima

Para cada uno de los casos hay que introducir diferentes niveles de ruido, desde niveles bajos hasta ir subiendolos progresivamente. Hay que tener en cuenta que en cada caso, el modelo se ve influenciado de una manera u otra, entonces estos niveles de ruido tienen diferente nivel en cada uno de ellos.

Hay que tener en cuenta que dependiendo del número de slices alto, menos capas tenemos en cada slice, y por tanto, para asegurarnos que realmente se afecta las capas habrá que seleccionar un porcentaje más alto


Se puede seleccionar el slice  que que quermos que se modifique
Con este parámetro seleccionamos la zona en la que queremos afectar a las activacionies o pesos
ees decir, slices = [slice_i, n_slices], con esto dividimos todas las capas en n_slices y afectamos a las capas que esten en slice_i




  |slice 0 | slice 1 | slice 2 | slice 3 | slice4

si se selecciona slice_i = n_slices se recorre desde slice_0,, hasta slice_n-1.

Luego existe el parámetro **percentage_layers_case1** que es un array de porcentajes. Para cada activación (LeakyRelu o Relu) (caso1) o peso (caso2) de cada slice, se calcula un número aleatorio y si es menor que el porcentaje correspondiente del array, esa capa sufrirá el efecto del bit flip. Por otro lado, como los pesos y activaciones son un conjunto de tensores, con el parámetro **percentage_tensor** se determina el porcentaje de tensores que se ven afectados de dicha capa.

## **Test case1**

Como dividimos las salidas de las funciones de activación en 3 bloques, se selecciona un porcentaje de tensores/parametros afectados para cada caso 
* Bloque 1: las funciones de activación tienen de orden $10^6$ parámetros, por tanto, una selección del 0.05\% de parámetros afectados es razonable.
*  Bloque 2: las funciones de activación tienen de orden $10^5$ y $10^4$, aquí se ha seleccionado un porcentaje de parámetros afectados más elevado, 0.5\%.
*  Bloque 3: las funciones de activación tienen de orden $10^4$ y $10^3$, en esta situación se selecciona un 5\% de parámetros afectados. 

In [3]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.3, 0.5, 0.7, 1] if case1 else [0]

#prueba 1
slices_case1 = [1,1]
interval_1 = (-32,2)
percentage_tensors_1 = 0.05

test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0])

In [ ]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.3, 0.5, 0.7, 1] if case1 else [0]

#prueba 1
slices_case1 = [1,1]
interval_1 = (-2,2)
percentage_tensors_1 = 0.05

test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0])

In [ ]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.5, 1] if case1 else [0]

#prueba 1
slices_case1 = [3,3]
interval_1 = (-32,2)
percentage_tensors_1 = 0.05

test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0])

In [ ]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.5, 1] if case1 else [0]

#prueba 1
slices_case1 = [3,3]
interval_1 = (-2,2)
percentage_tensors_1 = 0.05

test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0])

Test with case1: True, case2: False
File case1_results_noise_slices_3_interval1_-2_2_p_0.05.txt exists. Deleting...
Results will be saved in: case1_results_noise_slices_3_interval1_-2_2_p_0.05.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 1: Slice: [1, 3], Percentage: 0
############################################


Eval: Valid: : 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050553
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050553
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 1: Slice: [2, 3], Percentage: 0
############################################


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.02it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050551
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050551
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 1: Slice: [3, 3], Percentage: 0
############################################


...

random_index_array: 12391
x_flatten: 247808, Percentage: 0.05, Range: 12390
random_index_array: 12391
x_flatten: 247808, Percentage: 0.05, Range: 12390


Eval: Valid: : 100%|██████████| 2/2 [14:44<00:00, 442.37s/it]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.690      |8.436       |4.233       |2.986       |0.035       |
---------------------------------------------------
Valid: 	 mAP@50: 0.612910, mAP@75: 0.193978, mAP@90: 0.001536, Mean Loss: 15.690464
Valid: 	 mAP@50: 0.612910, mAP@75: 0.193978, mAP@90: 0.001536, Mean Loss: 15.690464
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 1: Slice: [1, 3], Percentage: 1
############################################


Eval: Valid: :   0%|          | 0/2 [00:00<?, ?it/s]

random_index_array: 198247
x_flatten: 3964928, Percentage: 0.05, Range: 198246


## **Test case 2**

In [1]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2 = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]

#prueba 4
slices_case2 = [1,1]
interval_2 = (-32,2)
percentage_tensors_2 = 0.0001

test_noise( case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2)

Test with case1: False, case2: True
Results will be saved in: case2_results_noise_slices_1_interval2_-32_2_p_0.0001.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 2: Slice: [1, 1], Percentage: 1
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
Number of layers affected: 34
Name: darknet.0.conv.weight, Shape: torch.Size([16, 3, 3, 3]), Flattened: torch.Size([432])
Random index: 1
Random index: 322 Exponential:  -15 
	Param: 0.029807521030306816, Bits: 00.00000111101000010111011100111
	Param: 0.029838038608431816, Bits: 00.00000111101000110111011100111
Number of weights affected: 1
Name: darknet.0.batchnorm.weight, Shape: torch.Size([16]), Flattened: torch.Size([16])
Random index: 1
Random index: 13 Exponential:  -23 
	Param: 0.2957314848899841

Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  3.45it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.890      |8.368       |4.376       |3.062       |0.084       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756833, mAP@75: 0.241908, mAP@90: 0.003144, Mean Loss: 15.890167
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
Number of layers affected: 34
Name: darknet.0.conv.weight, Shape: torch.Size([16, 3, 3, 3]), Flattened: torch.Size([432])
Random index: 1
Random index: 226 Exponential:  0 
	Param: 0.006456312723457813, Bits: 00.000000011010011100011110111101
	Param: 1.0064562559127808, Bits: 01.000000011010011100011110111101
Number of weights affected: 1
Name: darknet.0.batchnorm.weight, Shape: torch.Size([16]), Flattened: torch.Size([16])
Random index: 1
Random index: 4 Exponential:  -28 
	Para

Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  9.86it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
65.607      |38.831      |25.988      |0.368       |0.420       |
---------------------------------------------------
Valid: 	 mAP@50: 0.027778, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 65.607344
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
Number of layers affected: 34
Name: darknet.0.conv.weight, Shape: torch.Size([16, 3, 3, 3]), Flattened: torch.Size([432])
Random index: 1
Random index: 307 Exponential:  -13 
	Param: -0.08880772441625595, Bits: 10.000101101011110000011010011
	Param: -0.08868565410375595, Bits: 10.000101101011010000011010011
Number of weights affected: 1
Name: darknet.0.batchnorm.weight, Shape: torch.Size([16]), Flattened: torch.Size([16])
Random index: 1
Random index: 5 Exponential:  -18 
	Param:

Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  9.59it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
16.651      |8.452       |4.745       |3.432       |0.023       |
---------------------------------------------------
Valid: 	 mAP@50: 0.659380, mAP@75: 0.211895, mAP@90: 0.001096, Mean Loss: 16.651254
array_param_shape: 903104
Valid: 	 mAP@50: 0.481330, mAP@75: 0.211895, mAP@90: 0.001096, Mean Loss: 32.716255


In [1]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2 = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]

#prueba 4
slices_case2 = [1,1]
interval_2 = (-2,2)
percentage_tensors_2 = 0.003

test_noise( case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2)

Test with case1: False, case2: True
File case2_results_noise_slices_1_interval2_-2_2_p_0.003.txt exists. Deleting...
Results will be saved in: case2_results_noise_slices_1_interval2_-2_2_p_0.003.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 2: Slice: [1, 1], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  3.30it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050553
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.32it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050553
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
140.821     |96.290      |20.569      |23.340      |0.621       |
---------------------------------------------------
Valid: 	 mAP@50: 0.000778, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 140.820610
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.72it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
84.310      |40.369      |27.806      |13.655      |2.481       |
---------------------------------------------------
Valid: 	 mAP@50: 0.020016, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 84.309929
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  5.49it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
979.777     |425.307     |114.388     |425.094     |14.988      |
---------------------------------------------------
Valid: 	 mAP@50: 0.001343, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 979.776672
array_param_shape: 903104
Valid: 	 mAP@50: 0.007379, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 401.635737


In [3]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2 = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]

#prueba 4
slices_case2 = [3,3]
interval_2 = (-32,2)
percentage_tensors_2 = 0.001

test_noise( case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2)

Test with case1: False, case2: True
File case2_results_noise_slices_3_interval2_-32_2_p_0.001.txt exists. Deleting...
Results will be saved in: case2_results_noise_slices_3_interval2_-32_2_p_0.001.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 2: Slice: [1, 3], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.74it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050551
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.28it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050553
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
90.778      |59.233      |10.186      |20.227      |1.132       |
---------------------------------------------------
Valid: 	 mAP@50: 0.154916, mAP@75: 0.000535, mAP@90: 0.000000, Mean Loss: 90.778236
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.06it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
126.712     |70.175      |19.569      |34.549      |2.419       |
---------------------------------------------------
Valid: 	 mAP@50: 0.031718, mAP@75: 0.002735, mAP@90: 0.000133, Mean Loss: 126.712029
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.02it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
85.933      |44.127      |19.845      |21.515      |0.445       |
---------------------------------------------------
Valid: 	 mAP@50: 0.274964, mAP@75: 0.002480, mAP@90: 0.000000, Mean Loss: 85.932566
array_param_shape: 791424
Valid: 	 mAP@50: 0.153866, mAP@75: 0.002480, mAP@90: 0.000000, Mean Loss: 101.140944


In [2]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2 = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]

#prueba 4
slices_case2 = [3,3]
interval_2 = (-2,2)
percentage_tensors_2 = 0.003

test_noise( case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2)

Test with case1: False, case2: True
File case2_results_noise_slices_3_interval2_-2_2_p_0.003.txt exists. Deleting...
Results will be saved in: case2_results_noise_slices_3_interval2_-2_2_p_0.003.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar
############################################
Noise level 2: Slice: [1, 3], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.29it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050551
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.55it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
14.051      |7.427       |4.128       |2.466       |0.030       |
---------------------------------------------------
Valid: 	 mAP@50: 0.756015, mAP@75: 0.301328, mAP@90: 0.001536, Mean Loss: 14.050553
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
61.509      |38.100      |7.500       |14.801      |1.107       |
---------------------------------------------------
Valid: 	 mAP@50: 0.171196, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 61.508650
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.84it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
90.038      |54.971      |6.631       |25.730      |2.706       |
---------------------------------------------------
Valid: 	 mAP@50: 0.052603, mAP@75: 0.000154, mAP@90: 0.000000, Mean Loss: 90.038101
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model/35_YOLO_best.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  4.38it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
1499.595    |612.608     |36.998      |838.231     |11.759      |
---------------------------------------------------
Valid: 	 mAP@50: 0.002867, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 1499.595398
array_param_shape: 791424
Valid: 	 mAP@50: 0.075555, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 550.380716


# **Analisis para el modelo cuantizado**

## **Test case 1**

In [ ]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.3, 0.5, 0.7, 1] if case1 else [0]

#prueba 1
slices_case1 = [1,1]
interval_1 = (-8,2)
percentage_tensors_1 = 0.05

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0],
            folder_model = folder_model, model_name = model_name)           

In [ ]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.3, 0.5, 0.7, 1] if case1 else [0]

#prueba 1
slices_case1 = [1,1]
interval_1 = (-2,2)
percentage_tensors_1 = 0.05

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0],
            folder_model = folder_model, model_name = model_name)           

In [ ]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.5, 1] if case1 else [0]

#prueba 1
slices_case1 = [3,3]
interval_1 = (-32,2)
percentage_tensors_1 = 0.05

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0],
            folder_model = folder_model, model_name = model_name)           

In [ ]:
from eval_noise import test_noise
case1=True
case2=False
percentage_layers_case_1  = [0, 0.5, 1] if case1 else [0]

#prueba 1
slices_case1 = [3,3]
interval_1 = (-2,2)
percentage_tensors_1 = 0.05

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise( case1=case1, percentage_layers_case_1=percentage_layers_case_1, slices_case1 = slices_case1, interval_1 = interval_1, percentage_tensors_1=percentage_tensors_1,
            case2=case2, percentage_layers_case_2=[0], slices_case2 = [0,0],
            folder_model = folder_model, model_name = model_name)           

## **Test case 2**

In [5]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2  = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]


#prueba 4
slices_case2 = [1,1]
interval_2 = (-8,2)
percentage_tensors_2 = 0.001

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise(case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2,
            folder_model = folder_model, model_name = model_name)

Test with case1: False, case2: True
Results will be saved in: case2_results_noise_slices_1_interval2_-8_2_p_0.001.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar
############################################
Noise level 2: Slice: [1, 1], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.11it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110722
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.17it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110721
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
57.391      |33.037      |19.156      |4.316       |0.881       |
---------------------------------------------------
Valid: 	 mAP@50: 0.023246, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 57.390692
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.18it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
48.072      |18.790      |13.096      |15.699      |0.487       |
---------------------------------------------------
Valid: 	 mAP@50: 0.264201, mAP@75: 0.009347, mAP@90: 0.000000, Mean Loss: 48.071770
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.56it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
63.132      |25.274      |22.243      |15.351      |0.263       |
---------------------------------------------------
Valid: 	 mAP@50: 0.263158, mAP@75: 0.060008, mAP@90: 0.002924, Mean Loss: 63.131525
array_param_shape: 903104
Valid: 	 mAP@50: 0.183535, mAP@75: 0.060008, mAP@90: 0.002924, Mean Loss: 56.197996


In [3]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2  = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]


#prueba 4
slices_case2 = [1,1]
interval_2 = (-2,2)
percentage_tensors_2 = 0.003

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise(case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2,
            folder_model = folder_model, model_name = model_name)

Test with case1: False, case2: True
File case2_results_noise_slices_1_interval2_-2_2_p_0.003.txt exists. Deleting...
Results will be saved in: case2_results_noise_slices_1_interval2_-2_2_p_0.003.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar
############################################
Noise level 2: Slice: [1, 1], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.06it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110722
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.40it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110721
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
140.988     |47.025      |30.020      |63.296      |0.647       |
---------------------------------------------------
Valid: 	 mAP@50: 0.000199, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 140.988201
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.45it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
67.836      |42.645      |22.383      |2.455       |0.352       |
---------------------------------------------------
Valid: 	 mAP@50: 0.001754, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 67.836481
array_param_shape: 903104
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.74it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
139.405     |101.317     |13.784      |23.750      |0.554       |
---------------------------------------------------
Valid: 	 mAP@50: 0.008953, mAP@75: 0.000101, mAP@90: 0.000000, Mean Loss: 139.404736
array_param_shape: 903104
Valid: 	 mAP@50: 0.003636, mAP@75: 0.000101, mAP@90: 0.000000, Mean Loss: 116.076473


In [7]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2  = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]


#prueba 4
slices_case2 = [3,3]
interval_2 = (-8,2)
percentage_tensors_2 = 0.001

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise(case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2,
            folder_model = folder_model, model_name = model_name)

Test with case1: False, case2: True
File case2_results_noise_slices_3_interval2_-8_2_p_0.001.txt exists. Deleting...
Results will be saved in: case2_results_noise_slices_3_interval2_-8_2_p_0.001.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar
############################################
Noise level 2: Slice: [1, 3], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.86it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110721
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.34it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110722
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
20.885      |11.967      |6.852       |1.914       |0.151       |
---------------------------------------------------
Valid: 	 mAP@50: 0.506335, mAP@75: 0.077023, mAP@90: 0.000000, Mean Loss: 20.884714
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.35it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
23.149      |13.482      |6.410       |2.354       |0.902       |
---------------------------------------------------
Valid: 	 mAP@50: 0.502399, mAP@75: 0.063979, mAP@90: 0.000000, Mean Loss: 23.148706
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.92it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
22.719      |12.775      |7.642       |2.037       |0.265       |
---------------------------------------------------
Valid: 	 mAP@50: 0.483323, mAP@75: 0.096492, mAP@90: 0.000000, Mean Loss: 22.718988
array_param_shape: 791424
Valid: 	 mAP@50: 0.497353, mAP@75: 0.096492, mAP@90: 0.000000, Mean Loss: 22.250803


In [5]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2  = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]


#prueba 4
slices_case2 = [3,3]
interval_2 = (-2,2)
percentage_tensors_2 = 0.003

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise(case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2,
            folder_model = folder_model, model_name = model_name)

Test with case1: False, case2: True
File case2_results_noise_slices_3_interval2_-2_2_p_0.003.txt exists. Deleting...
Results will be saved in: case2_results_noise_slices_3_interval2_-2_2_p_0.003.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar
############################################
Noise level 2: Slice: [1, 3], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  2.12it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110720
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  6.52it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110721
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
229.705     |137.709     |21.151      |63.326      |7.519       |
---------------------------------------------------
Valid: 	 mAP@50: 0.066110, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 229.705017
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.05it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
137.388     |106.538     |5.531       |24.265      |1.054       |
---------------------------------------------------
Valid: 	 mAP@50: 0.144003, mAP@75: 0.002831, mAP@90: 0.000000, Mean Loss: 137.387810
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  6.58it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
61.441      |33.350      |4.917       |20.517      |2.657       |
---------------------------------------------------
Valid: 	 mAP@50: 0.162992, mAP@75: 0.000975, mAP@90: 0.000000, Mean Loss: 61.440725
array_param_shape: 791424
Valid: 	 mAP@50: 0.124368, mAP@75: 0.000975, mAP@90: 0.000000, Mean Loss: 142.844517


In [6]:
from eval_noise import test_noise
case1=False
case2=True
percentage_layers_case_2  = [0, 0.3, 0.5, 0.7, 1] if case2 else [0]


#prueba 4
slices_case2 = [3,3]
interval_2 = (-4,2)
percentage_tensors_2 = 0.003

folder_model = '6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4'
model_name = 'YOLO_opt.pth.tar'
test_noise(case1=case1, percentage_layers_case_1=[0], slices_case1 = [0,0],
           case2=case2, percentage_layers_case_2=percentage_layers_case_2, slices_case2 = slices_case2, interval_2 = interval_2, percentage_tensors_2= percentage_tensors_2,
            folder_model = folder_model, model_name = model_name)

Test with case1: False, case2: True
Results will be saved in: case2_results_noise_slices_3_interval2_-4_2_p_0.003.txt
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar
############################################
Noise level 2: Slice: [1, 3], Percentage: 0
############################################
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.52it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110722
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  8.04it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
15.111      |7.942       |4.850       |2.253       |0.065       |
---------------------------------------------------
Valid: 	 mAP@50: 0.746276, mAP@75: 0.258633, mAP@90: 0.001629, Mean Loss: 15.110720
array_param_shape: 0
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


...

---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
40.288      |20.976      |6.384       |11.517      |1.411       |
---------------------------------------------------
Valid: 	 mAP@50: 0.474761, mAP@75: 0.065413, mAP@90: 0.000000, Mean Loss: 40.288413
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  7.74it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
187.176     |125.127     |11.325      |46.299      |4.426       |
---------------------------------------------------
Valid: 	 mAP@50: 0.149136, mAP@75: 0.039738, mAP@90: 0.000302, Mean Loss: 187.176373
array_param_shape: 791424
=> Loading checkpoint
Model loaded: tinyissimoYOLO/NAdam_None_Airbus_256_BATCH_32_LR_0.0005/model_opt/6-TIC_5_sims_max_steps_100_slices_5_interval_4_10_degredation_0/sim_4/YOLO_opt.pth.tar


Eval: Valid: : 100%|██████████| 2/2 [00:00<00:00,  6.61it/s]


---------------------------------------------------
-------------Loss Summary eval Valid--------------
Total Loss  |Loss Coord  |Conf Loss   |No Obj Loss |Class Loss  |
439.716     |312.100     |10.738      |113.177     |3.701       |
---------------------------------------------------
Valid: 	 mAP@50: 0.006939, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 439.716042
array_param_shape: 791424
Valid: 	 mAP@50: 0.210279, mAP@75: 0.000000, mAP@90: 0.000000, Mean Loss: 222.393609


In [2]:

def convert_fractional_to_binary(number, precision=32):
    fractional_part = number - int(number)
    binary_fractional_part = []
    while fractional_part != 0 and len(binary_fractional_part) < precision:
        fractional_part *= 2
        bit = int(fractional_part)
        binary_fractional_part.append(abs(bit))
        fractional_part -= bit
    return binary_fractional_part

def fractional_bits_required(number, precision=10):
  
    # The number of bits required is the length of the binary_fractional_part list
    return len(convert_fractional_to_binary(number, precision=precision))

def convert_fractional_binary_to_fractional(int_value, binary_fractional_part):
    fractional_part = abs(int_value)
    for i in range(len(binary_fractional_part)):
        fractional_part += abs(binary_fractional_part[i]) * 2**(-(i+1))
    return fractional_part

def convert_int_binary_to_int(binary_int_part):
    fractional_part = 0
    for i in range(len(binary_int_part)):
        fractional_part += binary_int_part[i] * 2**(i)
    return fractional_part




def string_bit(sign_bit, int_bits, frac_bits):
    
    bits = f'{1 if sign_bit==-1 else 0}'
    for int_bit in int_bits:
        bits = f'{bits}{int_bit}'
    bits = f'{bits}.'
    for frac_bit in frac_bits:
        bits = f'{bits}{frac_bit}'
    return bits

In [12]:
bin_frac = convert_fractional_to_binary( 0.37078234553337097)

In [35]:
bin_frac[:4]

[0, 1, 0, 1]

In [37]:
convert_fractional_binary_to_fractional(0, bin_frac[:4])

0.3125